<a href="https://colab.research.google.com/github/iyushanand/AI_pair_programmer/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing the needed libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re # regular expression
import string

IMPORTING DATASET

In [2]:
df_fake = pd.read_csv("/content/Fake.csv")
df_true = pd.read_csv("/content/True.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/Fake.csv'

In [ ]:
df_fake.head()

In [ ]:
df_true.head(5)

In [ ]:
# Inserting a column class "Class" as target feature
df_fake["class"] = 0
df_true["class"] = 1

In [ ]:
df_fake.shape , df_true.shape

In [ ]:
# removing the last ten rows for manual testing
df_fake_manual_testing = df_fake.tail(10)
for i in range(23480,23470,-1): # backward 10 rows
  df_fake.drop([i],axis=0,inplace=True)

# same for df true manual testing last 10 elements
df_true_manual_testing = df_true.tail(10)
for i in range(21416,21406,-1): # backward 10 rows
  df_true.drop([i],axis=0,inplace=True)

In [ ]:
df_fake.shape , df_true.shape # after removing the 10 rows for manual testing

In [ ]:
# again inserting a column class in these manual testing
df_fake_manual_testing["class"] = 0
df_true_manual_testing["class"] = 1

In [ ]:
# checking wheather inserted or not
df_fake_manual_testing.head(10)

In [ ]:
df_true_manual_testing.head(10)

In [ ]:
df_manual_testing = pd.concat([df_fake_manual_testing,df_true_manual_testing],axis=0) # made to one
df_manual_testing.to_csv("manual_testing.csv") # changed to csv file


Merging True and Fake Dataframes

In [ ]:
df_merge = pd.concat([df_fake,df_true],axis = 0 ) # fake and true merged
df_merge.head(10)


In [ ]:
df_merge.columns  # fetch the columns in merge data fram that we did earlier

####Removing Unwanted Columns

In [ ]:
df = df_merge.drop(["title","subject","date"],axis=1) # removed unwanted columns

In [ ]:
df.isnull().sum() # checking the number of null now

####Random Shuffling the Dataframe

In [ ]:
df = df.sample(frac=1)

In [ ]:
df.head()

In [ ]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [ ]:
df.columns

In [ ]:
df.head()

###Creating a function to process the text

In [ ]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
df["text"] = df["text"].apply(wordopt)

#### Defining dependant and independant variable

In [ ]:
x = df["text"]
y = df["class"]

####Splitting Training and Testing

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

#### Convert Text to Vectors

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

###Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train,y_train)

In [ ]:
pred_lr=LR.predict(xv_test)

In [ ]:
LR.score(xv_test, y_test)

In [ ]:
print(classification_report(y_test, pred_lr))

### Decision Tree Classification

In [ ]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

In [ ]:
pred_dt = DT.predict(xv_test)

In [ ]:
DT.score(xv_test, y_test)

In [ ]:
print(classification_report(y_test, pred_dt))

### Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

In [ ]:
pred_gbc = GBC.predict(xv_test)

In [ ]:
GBC.score(xv_test, y_test)

In [ ]:
print(classification_report(y_test,pred_gbc))

### Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

In [ ]:
pred_rfc = RFC.predict(xv_test)

In [ ]:
RFC.score(xv_test, y_test)

In [ ]:
print(classification_report(y_test, pred_rfc))

### Model Testing

In [ ]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"

def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]),                                                                                                       output_lable(pred_DT[0]),
                                                                                                              output_lable(pred_GBC[0]),
                                                                                                              output_lable(pred_RFC[0])))

In [ ]:
news = str(input())
manual_testing(news)


In [ ]:
news = str(input())
manual_testing(news)

In [ ]:
news = str(input())
manual_testing(news)